In [18]:
# 主要是对ajob表进行去重
import pandas as pd
import datetime
import time
pd.options.mode.chained_assignment = None  # default='warn'

In [25]:
before_day = ['180602', '180603', '180604', '180605', '180606', '180607', '180608', '180609', '180610',\
           '180611', '180612', '180613', '180614', '180615', '180616', '180617', '180618', '180619', '180620',\
           '180621', '180622', '180623', '180624', '180625', '180626', '180627', '180628', '180629', '180630',\
           '180701', '180702', '180703', '180704', '180705']

day_all = ['180716', '180717', '180718', '180719', '180720', '180721', '180722', '180723', '180724', '180725',\
           '180726', '180727', '180728', '180729', '180730', '180731', '180801', '180802', '180803', '180804',\
           '180805', '180806', '180807', '180808', '180809', '180810', '180811', '180812', '180813', '180814',\
           '180815', '180816', '180817', '180818', '180819', '180820', '180821', '180822',  '180826',\
           '180827', '180828', '180829', '180830', '180831', '180901', '180902', '180903', '180904', '180905',\
           '180906', '180907', '180908', '180909', '180910', '180911', '180912', '180913', '180914', '180915',\
           '180916', '180917', '180918', '180919', '180920', '180921', '180922', '180923', '180924', '180925', \
            '180926', '180927', '180928', '180929', '180930', '181001', '181002', '181003', '181004', '181005', \
           '181006', '181007', '181008', '181009', '181010', '181011', '181012', '181013', '181014', '181015', \
           '181016', '181017', '181018', '181019', '181020', '181021',\
           '181022', '181023', '181024', '181025', '181026', '181027', '181028', '181029', '181030', \
           '181031', '181101', '181102', '181103', '181104', '181105', '181106', '181107', '181108', '181109',\
           '181110', '181111']
pred_day = ['181105', '181106', '181107', '181108', '181109','181110', '181111']

def get_avg_run_stat_mint(s):
    # 把时间转换成10进制数据  60进制-->100进制
    if s is None:
        return None
    else:
        h = s//10000
        m = (s%10000)//100
        
        s = s%100 #不用精确到秒
        return h*60+m+s/60


In [26]:
#for day in day_all:
for day in before_day:
    
    #day = '180603'   #调试用
    
    
    day_4_hour = int('20'+day +'040000')
    day_int = int(day)
    day_begain = int('20'+day+'000000')

    #---------------------------------------------------------------------------------------------------
    if day in pred_day:
        ajob = pd.read_csv(r'E:\jupyter\CTM\data_src\ajob\ajob_%s_0_.txt'%day,low_memory=False)
    else:
        ajob = pd.read_csv(r'E:\jupyter\CTM\data_src\ajob\ajob_%s_0.txt'%day,low_memory=False)
    ajob = ajob[ajob['order_time']<=day_4_hour]
    #ajob = ajob[ajob.end_time<= day_after_end]
    ajob = ajob[ajob.odate==day_int]
    ajob = ajob[ajob.task_type!='SMART Table']
    ajob = ajob.sort_values(by='end_time')
    
    
    ajob.drop_duplicates('job_name', keep='first', inplace=True)
    ajob['server'] = ajob['server'].apply(str)
    ajob['order_id'] = ajob[['order_id','server']].apply(lambda x:x[0]+x[1],axis =1)
    
    ajob['avg_runtime'] = ajob['avg_runtime'].apply(get_avg_run_stat_mint)
    ajob['avg_start_time'] = ajob['avg_start_time'].apply(get_avg_run_stat_mint)
    
    
    ajob.drop('order_d',axis=1,inplace=True)
    ajob.drop('interval',axis=1,inplace=True)
    ajob.drop('odate',axis=1,inplace=True)
    ajob.drop('confirm_flag',axis=1,inplace=True)
    ajob.drop('job_id',axis=1,inplace=True)
    ajob.drop('next_time',axis=1,inplace=True)
    ajob.drop('to_time',axis=1,inplace=True)
    ajob.drop('delete_flag',axis=1,inplace=True)
    ajob.drop('rerun_counter',axis=1,inplace=True)
    ajob.drop('prev_odate_rerun_counter',axis=1,inplace=True)
    ajob.drop('cpu_time',axis=1,inplace=True)
    ajob.drop('cpu_id',axis=1,inplace=True)
    
    dat_start = datetime.datetime.strptime(str(day_begain),'%Y%m%d%H%M%S')
    def f_get_mint_bef(x):
        if x ==-1:
            return None
        else:
            dat_t = datetime.datetime.strptime(str(x),'%Y%m%d%H%M%S')
            dt = (dat_t-dat_start).total_seconds()//60
            return dt
    ajob['edt'] = ajob['end_time'].fillna(-1).apply(int).apply(f_get_mint_bef)
    
    ajob = ajob.round(3)
    ajob.to_csv(r'E:\jupyter\CTM\data_src\ajob_0\ajob_%s_0.csv'%day,index=False)
    
    print(day,end=' ')

180602 180603 180604 180605 180606 180607 180608 180609 180610 180611 180612 180613 180614 180615 180616 180617 180618 180619 180620 180621 180622 180623 180624 180625 180626 180627 180628 180629 180630 180701 180702 180703 180704 180705 

In [13]:
ajob['end_time'].fillna(-1).tail()

260556   -1.0
260565   -1.0
260567   -1.0
261171   -1.0
261203   -1.0
Name: end_time, dtype: float64